In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
# removing NA 
heart = heart.dropna()

In [14]:
# defining input and target variables 
x = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP','BMI', 'heartRate', 'glucose']]
y = heart['TenYearCHD']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [15]:
# scaling the data
scaler = MinMaxScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_train.columns)

In [16]:
# identifying the importaint variables with lasso
lasso_md = LassoCV().fit(x_train, y_train)

## extracting best labda
lasso_lambda = lasso_md.alpha_

# fitting lasso with best labda
lasso_md = Lasso(alpha = lasso_lambda).fit(x_train, y_train)
lasso_md.coef_

array([ 0.07328869,  0.27055981,  0.04687772,  0.        ,  0.41166477,
        0.        , -0.        ,  0.37970591])

In [17]:
#defining variables for logistic
x_train_logit = x_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
x_test_logit = x_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

#building logistic regression model 
logit_md = LogisticRegression().fit(x_train_logit, y_train)

# predicting on test
logit_pred = logit_md.predict_proba(x_test_logit)[:,1]

# changing liklihoods to labels
logit_label = np.where(logit_pred < .1, 0, 1)

# computing recall
logit_recall = recall_score(y_test, logit_label)
print('recall for logistic model:', logit_recall)

recall for logistic model: 0.8061224489795918
